<img style="float: left;" src="resources/made.jpg" width="35%" height="35%">

# Академия MADE
## Семинар 10: Metric learning
Эдуард Тянтов, руководитель направления машинного обучения в Почте и Портале Mail.ru

В рамках семинар обучим модель распознавания лиц, используя несколько техник Metric Learning, 
на небольшом подмножестве MSCeleb.
Данные можно найти по https://cloud.mail.ru/public/33FM/ucXu3rW7n/ (это небольшая подвыборка из MSCeleb: 50k элементов)
Код написан на python 3.7.
Для выполнения работы нужны следующие пакеты:
* torch, torchvision (conda install pytorch torchvision cudatoolkit=9.2 -c pytorch)
* opencv, matplotlib (conda install -c conda-forge opencv matplotlib)
* pandas, jupyter (conda install pandas jupyter)
* faiss (conda install -c pytorch faiss-cpu)

# План
* Смотрим на данные
* обучаем сеть c Softmax
* Center-loss
* Arcface
* Оцениваем все 3 метода по валидации
* Тестируем на faiss/hnsw лучший из них

In [ ]:
# imports
import os
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import datetime, time
from utils import save_object, load_object
print(torch.__version__)
from IPython.display import Image
device = torch.cuda.is_available() and 'cuda' or 'cpu'
num_devices = torch.cuda.device_count()
print ('num_devices=%d' % num_devices)
# set only one device for now
if device == 'cuda':
   device = 'cuda:0'

In [ ]:
# data paths
# пропишите два путя из распакованного архива с данными
data_path = '/home/tyantov/small_celeb/'
test_data_path = '/home/tyantov/test_small_celeb/'
#
pretrained_arc = 'resources/arc_resnet18_512_16_epochs_checkpoint.pth.tar'
pretrained_softmax = 'resources/softmax_resnet18_512_10_epochs_checkpoint.pth.tar'
pretrained_centerloss = 'resources/centerloss_resnet18_512_10_epochs_checkpoint.pth.tar'
lfw_path = 'resources/lfw.bin' # lfw test dataset dump

# sanity checks
assert os.path.exists(data_path)
assert os.path.exists(test_data_path)


# Данные
Давайте посмотрим, что из себя представляют данные из MSCeleb.
Датасет представляет из себя набор папок, в каждой папке набор вырезанных лиц одного человека.
``
>$ ls small_celeb | head -5

100 
1000
10000
10001
10002``
``
>$ ls small_celeb/100/100

2746.jpg  2747.jpg  2748.jpg  2749.jpg  2750.jpg  2751.jpg  2752.jpg  2753.jpg  2754.jpg  2755.jpg  2756.jpg  2757.jpg  2758.jpg  2759.jpg  2760.jpg  2761.jpg  2762.jpg  2763.jpg  2764.jpg``

Давайте посмотрим как выглядят данные, по персонам.

In [ ]:
persons = os.listdir(data_path)
person = persons[np.random.randint(0,len(persons)-1)]
person_folder = os.path.join(data_path, person, person)
files = os.listdir(person_folder)

SAMPLE_SIZE = len(files)
NUM_COLS = 5
NUM_ROWS = SAMPLE_SIZE // NUM_COLS + int(SAMPLE_SIZE % NUM_COLS != 0)
plt.figure(figsize=(20, 2 * NUM_ROWS))

for i, _file in enumerate(files):
    image = cv2.imread(os.path.join(person_folder, _file))
    text = _file
    plt.subplot(NUM_ROWS, NUM_COLS, i+1)
    plt.imshow(image[:, :, ::-1])
    plt.title(text)
    plt.axis("off")

plt.tight_layout()
plt.show()

num_classes = len(persons)

# Тренировка

Сначала создаим датасет. Начнем с аугментаций.
Cutout - отличная аугментация, которую вы возможно не знали.
Суть простая - зануляем пиксили в рандомно регионе заданного размера, имитируя таким образом
occlusion.

In [ ]:
class Cutout(object):
    """Аугментация, которая вырезает кусок заданного размера из картинки
    статья: https://arxiv.org/pdf/1708.04552.pdf"""
    def __init__(self, size=8):
        self.size = size

    def __call__(self, tensor):
        _, h, w = tensor.shape
        y = np.random.randint(0, h)
        x = np.random.randint(0, w)
        tensor[:, y - self.size:y + self.size, x - self.size:x + self.size] = 0

        return tensor

Выглядит при аугментации это след. образом:

<img style="float: left;" src="resources/cutout.png" width="35%" height="35%">

Зададим остальные трансформации:

In [ ]:
# импорт стандартных библиотек для трансформаций и датасета
import torchvision.transforms as transforms
import torchvision.datasets
# inbuilt cudnn auto-tuner to find the best algorithm to use for your hardware.
if device == 'gpu':
    import torch.backends.cudnn as cudnn
    cudnn.benchmark = True

normalize = transforms.Normalize(mean=[0.5, 0.5, 0.5],
                                     std=[0.5, 0.5, 0.5])

train_transform = transforms.Compose([
    # вертикальный флип нам не подойдет, если нужно распознавать такие фотки,
    # то на детекторе надо предсказывать перевернутость головы, что может породить ошибки при инференсе
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    normalize,
    Cutout(size=14) # по сути у нас две аугментации всего
    ])
test_transform = transforms.Compose([transforms.ToTensor(), normalize]) # при инференсе только нормализуем

# используем стандартный датасет от пайторча
#the images are arranged in this way:
#         root/dog/xxx.png
#         root/dog/xxy.png
#         root/dog/xxz.png
#
#         root/cat/123.png
#         root/cat/nsdf3.png
#         root/cat/asd932_.png
train_dataset = torchvision.datasets.ImageFolder(data_path, train_transform)
test_dataset = torchvision.datasets.ImageFolder(test_data_path, test_transform)

print('The size of the train dataset:', len(train_dataset))
print('The size of the test dataset:', len(test_dataset))

# И финально зададим даталоадер

batch_size = 160 # ставьте 40 и сеть при обучении займет 3GB, 160 - 10.5Gb
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True,
    num_workers=4, pin_memory=True, drop_last=True, sampler=train_sampler)
test_loader = torch.utils.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False,
    num_workers=4, pin_memory=True, drop_last=False, sampler=train_sampler)

# Обратите внимание, что shuffle & drop_last на инференсе False
# На что влиет num_workers ? Как его подбирать ? Как зависит от дисков?

Перейдем к различным loss'ам.
Сначала обучим сеть с помощью софтмакса
Нам надо сделать небольшую модификацию линейного слоя, чтобы он всегда
нормализовал веса и входной тензор. Для того, чтобы работать в на единичной сфере.
Чем это удобно кстати ?

<img style="float: left;" src="resources/softmax_norm.jpg" width="35%" height="35%">


In [ ]:
from torch.nn import functional as F
from torch import nn
class LinearScoring(nn.Linear):
    def __init__(self, in_features, out_features):
        super(LinearScoring, self).__init__(in_features, out_features, bias=False)

    def forward(self, input, target=None):
        "The only difference is normalization of the input and weights"
        input = F.normalize(input, p=2, dim=1)
        scores = F.linear(input, F.normalize(self.weight, p=2, dim=1))
        return scores

In [ ]:
# Здесь функции для инициализации разных лоссов
import torch.nn as nn
from models.scoring import ArcFaceScoring
from models.center_loss import CenterLoss
def init_arcface_scoring(emb_size, num_classes, m, s):
    scoring = ArcFaceScoring(m, s, in_features=emb_size, out_features=num_classes, device=device)
    criterion = nn.CrossEntropyLoss()
    return scoring, criterion

def init_softmax(emb_size, num_classes):
    scoring = LinearScoring(emb_size, num_classes)
    criterion = nn.CrossEntropyLoss()
    return scoring, criterion

def init_centerloss(emb_size, num_classes):
    scoring, criterion = init_softmax(emb_size, num_classes)
    center_loss = CenterLoss(num_classes, emb_size)
    return scoring, criterion, center_loss

In [ ]:
# params
center_loss, optimzer4center = None, None
cl_loss_weight = 5e-3 # вес при лоссе относительно CE
head_size = 7 * 7 * 512 # size of the last resnet layer's output
emb_size = 512
lr, center_lr = 0.01, 0.1
momentum=0.9
weight_decay = 0.0005
# Init scoring
#scoring, criterion = init_arcface_scoring(emb_size, num_classes, m=0.5, s=64.0)
scoring, criterion = init_softmax(emb_size, num_classes)
#scoring, criterion, center_loss = init_centerloss(emb_size, num_classes)
# Создадим модель и оптимизатор

from models.resnet import resnet
model = resnet(18, num_classes, emb_size=emb_size, scoring=scoring, head_size=head_size)

#model = torch.nn.parallel.DataParallel(model.to(device))
model = model.to(device)
criterion = criterion.to(device)

optimizer = torch.optim.SGD(model.parameters(), lr, momentum=momentum, weight_decay=weight_decay)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, patience=2, verbose=True)

if center_loss is not None:
    center_loss = center_loss.to(device)
    optimzer4center = torch.optim.SGD(center_loss.parameters(), lr =center_lr)
# TODO change lr over time scheduler
# <Изучаем модель в models/resnet.py>
print(model)

## Валидация
Валидироваться будем на стандартном небольшом тесте LFW (http://vis-www.cs.umass.edu/lfw/)
У нас есть список пар фотографий и список соответствий is_same_list: lfw.py#L224
в issame_list хранится информация о том, являются два лица из пары одним и тем же человеком или нет
мы находим эмбединг для лица (прогоняем два раза для лица и отзеркаленного лица и усредням)
далее для разных трешхолдов проверяем, что два заданных лица одного и того же человека считаются нашей сеткой одной парой:
lfw.py#L157

Обученные модели там выбивают примерно 99.7

In [ ]:
# validate

from eval.loader import load_bin
val_set = load_bin(lfw_path, (112, 112))

In [ ]:
# Traning routine
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    
from utils import AverageMeter, save_checkpoint

def train(train_loader, model, criterion, optimizer, epoch, start_epoch, print_freq=100):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    avg_loss = AverageMeter()

    end = time.time()
    for i, (input, target) in enumerate(train_loader):
        data_time.update(time.time() - end)
        input = input.to(device)
        target = target.to(device)

        emb, scores, losses = model(input, target)
        loss = criterion(scores, target)

        if center_loss is not None:
            cl_loss = center_loss(target, emb)
            loss += cl_loss.sum() * cl_loss_weight
            optimzer4center.zero_grad()

        avg_loss.update(loss.item(), input.size(0))

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        if optimzer4center is not None:
            optimzer4center.step()
        batch_time.update(time.time() - end)
        end = time.time()

        if i % print_freq == 0:
            line = 'Epoch: [{0}][{1}/{2}]\t' \
                   'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t' \
                   'Data time {data_time.val:.3f} ({data_time.avg:.3f})\t' \
                   'Loss {loss.val:.4f} ({loss.avg:.4f})\t'.format(
                epoch, i + start_epoch, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=avg_loss)
            print(line)


# train
from eval.lfw import test
start_epoch, epoch_duration, best_prec1 = 0, 0, 0
num_epochs = 10

checkpoint_path = 'checkpoints/'
if not os.path.exists(checkpoint_path):
    os.mkdir(checkpoint_path)

model.train()
for epoch in range(start_epoch, num_epochs):
    start_time = datetime.datetime.now()
    print("Previous epoch duration: %s" % epoch_duration)
    print("Current epoch start time:", start_time)

    train(train_loader, model, criterion, optimizer, epoch, start_epoch)
    # validate
    acc1, std1, prec1, std2, _xnorm, embeddings_list = test(val_set, model, batch_size, device=device)
    print('val lfw acc %1.5f+-%1.5f , acc2 %1.5f+-%1.5f \n' % (acc1, std1, prec1, std2))

    scheduler.step(prec1)

    end_time = datetime.datetime.now()
    epoch_duration = end_time - start_time

    # save
    is_best = prec1 > best_prec1
    best_prec1 = max(prec1, best_prec1)
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': 'resnet18',
        'state_dict': model.state_dict(),
        'best_prec1': best_prec1,
        'optimizer' : optimizer.state_dict(),
    }, is_best, os.path.join(checkpoint_path, 'checkpoint'))

print('Training ended, best pre1', best_prec1)

Давайте провалидируем модель

In [ ]:
LOAD = True
torch.cuda.empty_cache()
from utils import load_checkpoint

if LOAD:
    print('Loading trained for 10 epochs model')
    if isinstance(model.scoring, ArcFaceScoring):
        load_checkpoint(model, pretrained_arc)
        print(open('resources/arcface_train_log.txt', 'r').read())
    elif center_loss is not None:
        load_checkpoint(model, pretrained_centerloss)
        print(open('resources/centerloss_train_log.txt', 'r').read())
    else:
        load_checkpoint(model, pretrained_softmax)
        print(open('resources/softmax_train_log.txt', 'r').read())
else:
    load_checkpoint(model, os.path.join(checkpoint_path, 'checkpoint_best.pth.tar'))

acc1, std1, prec1, std2, _xnorm, embeddings_list = test(val_set, model, batch_size, device=device)
print('lfw acc %1.5f+-%1.5f , acc2 %1.5f+-%1.5f' % (acc1, std1, prec1, std2))

Напомню разумный максимум 99.7 - годится для замеров на небольшом датасете
для больших датасетов скор дорастает до максимального значения и дальше шумит
нужно использовать другую более трудную валидацию - например, Megaface

## Center loss
Давайте посмотрим на center loss
Напомню идею:


<img style="float: left;" src="resources/center_loss_idea.png" width="100%" height="100%">

По сути мы используем два лосса:
 - дискриминационный (softmax), чтобы обеспечить разделяемость
 - стягивающий для компактности (center loss)


<img style="float: left;" src="resources/center_loss_arch.png" width="100%" height="100%">

<Смотрим и разбираем код в models/scoring.py>

В Backprop вычисляется вот эта формула, хотя строго говоря можно было оставить автограду посчитать ее:


<img style="float: left;" src="resources/center_loss_grad.png" width="40%" height="40%">

## Arcface

Перейдем к Arcface.
Напомню основная идея A-Softmax лоссов - оперировать на единичной окружности, где за разделимость
отвечает угол между векторами (весами линейного слоя) классов.


<img style="float: left;" src="resources/arcface_idea.png" width="100%" height="100%">

Для этого мы усложняем задачу модели для элементов одного класса, занижая скор по
формуле.

<img style="float: left;" src="resources/arcface_reduce.png" width="100%" height="100%">
<img style="float: left;" src="resources/arcface_formula.png" width="100%" height="100%">

<Смотрим и разбираем код в models/scoring.py>

# Search indices
Давайте рассмотрим HNSW и Faiss.

Для начала давайте посчитаем тестовые embeddings

In [ ]:
# compute embeddings for test set
model.eval()

img_label_list = test_loader.dataset.imgs # img names & labels
embeddings = np.zeros((len(img_label_list), emb_size), dtype=np.float32)
targets = np.array([w[1] for w in img_label_list])
with torch.no_grad():
    for i, (input, target) in enumerate(test_loader):
        input = input.to(device)
        embed, _, _ = model(input, target=None, calc_scores=False)
        embed = embed.to('cpu').numpy() # shape batch_size x emb_size
        embeddings[i*batch_size:(i+1)*batch_size,:] = embed
print('Test_embeddings are ready', embeddings.shape)

Т.к. индексы AKNN предназначены для больших данных (иначе можно перебором все делать), то я прогнал 200k картинок из
MSceleb (не из трейна), чтобы на них посмотреть какие у нас получаться метрики и поиграться с параметрами индексов.
Использовал для прогона модель arcface, которая загружается выше.

In [ ]:
BIG_INDEX_ON = True
# load bigger numbers
if BIG_INDEX_ON:
    embeddings, targets = load_object('resources/msceleb_emb.dump')
    img_label_list = load_object('resources/msceleb_names.dump')

# exclude M photos of each person
import numpy as np
M = 1
prev, cnt = -1, 0
masks = np.zeros((embeddings.shape[0],), dtype=np.bool)
for i, v in enumerate(targets):
    is_test = (v != prev and cnt < M)
    masks[i] = is_test
    if is_test:
        cnt += 1
    else:
        prev, cnt = v, 0
train_embeddings, train_targets = embeddings[np.invert(masks)], targets[np.invert(masks)]
test_embeddings, test_targets = embeddings[masks], targets[masks]
print('Shape of embeddings total/train/test', embeddings.shape, train_embeddings.shape, test_embeddings.shape)

## Faiss

<img style="float: left;" src="resources/faiss_ivf.png" width="100%" height="100%">
Два основных параметра, которыми мы рулим:

- кол-во центройдов. Определяет на сколько частей мы бьем наше пространство
* если их мало, то будет долгий поиск, т.к. почти фулл скан нужно сделать
- чем больше центройдов, тем дольше строить индекс
- nprobes - кол-во проб (т.е. поисков по центройдам), при =1 берем только самый ближайший, но тогда будет низкая точность AKNN
 * линейно зависит время поиска

In [ ]:
# Будем мерить rank1 и засекать время
def evaluate(_index, test_embeddings):
    start = time.time()
    distances, items = _index.search(test_embeddings, 1)
    print('Timing', time.time() - start)

    scores = []
    for i in range(items.shape[0]):
        closest_idx, closest_dist = items[i, 0], distances[i, 0]
        scores.append(int(test_targets[i] == train_targets[closest_idx]))
    rank1 = sum(scores)/items.shape[0]
    print ('Rank1', rank1)
    return rank1

In [ ]:
import faiss
from utils import save_object, load_object

num_centroids = 32 # 32 is enough, the more - more expensive
faiss_algo = 'IVF%d,Flat' % (num_centroids,) # inverted files with so many centroids
print('Сreate faiss index')
index = faiss.index_factory(emb_size, faiss_algo)
index.verbose = True


# Training: по сути треним центройды и Product Quantizer
print('train', train_embeddings.shape, train_embeddings.dtype)
assert not index.is_trained
start = time.time()
index.train(train_embeddings)
print('Construction done', time.time() - start)

# А теперь загружаем данные в индекс, если будем добавлять новые данные, не факт,
# что обученные параметры будут эффективно искать по ним
#  Но если распределение не меняется то перетренировывать его не надо.
print('Add data to index')
index.add(train_embeddings)
ps = faiss.ParameterSpace()
ps.initialize(index)
index.search_type = faiss.IndexPQ.ST_PQ # устанавливаем тип индекса с квантизацией
print('Ready to query')

Сначала запустим фуллскан, чтобы понять какой rank1 максимальный

In [ ]:
# так как это обычный фуллскан, то "тренировки" нет
index_f = faiss.IndexFlatL2(emb_size)
index_f.add(train_embeddings)
evaluate(index_f, test_embeddings)

In [ ]:
index.nprobe = 5 # 1 - bad, 5 - ok
print('IVF, Num_centroids', num_centroids, 'probes', index.nprobe)
evaluate(index, test_embeddings)


### HNSW
<img style="float: left;" src="resources/hnsw.png" width="50%" height="50%">
Напомню, что алгоритм графовый, иерархичный. Основан на тесном мире.
Что важно:

- при построении индекса нужно чтобы элементы сэмплировались случайно.
 * Самые длинные ребра образуются на первых итерациях. Они позволяют быстро перемещаться по графу. Если семлировать из кластеров то длинных ребер не будет.
- построение: векторное представление добавляем в граф, чем больше efConstruction (длина списка), тем точнее вставляем в граф
 * из этого списка мы ищем M лучших соседей и проводим M ребер
- efSearch - Кол-во элементов в списке минимальных обнаруженных дистанций.
  В нем хранятся упорядоченные дистанции. Наибольшие дистанции вытесняются.
  В тот момент когда список не обновился мы считаем что нашли ближайшего соседа


In [ ]:
index_h = faiss.IndexHNSWFlat(emb_size, 32)
# длина списка вовремя построения
index_h.hnsw.efConstruction = 32 # 40 - is enough
#
start = time.time()
index_h.verbose = True
index_h.add(train_embeddings)
print('Construction HNSW done', time.time() - start)

In [ ]:
# длина списка вовремя поиска
index_h.hnsw.efSearch = 8 # 32 almost
print('HNSW, efConstruction=%d, efSearch=%d' % (index_h.hnsw.efConstruction, index_h.hnsw.efSearch))
evaluate(index_h, test_embeddings)

Сравнение на миллионе
https://github.com/facebookresearch/faiss/wiki/Indexing-1M-vectors
<img style="float: left;" src="resources/index_comp.png" width="100%" height="100%">

